In [ ]:
# 우선 "ifg@"라는 문자가 중간중간에 삽입되어있다.
# 또, 년 월 일 시간 이율(숫자) 카톡ID 연락처 택배송장번호 SMS인증번호 카드번호
# 숫자로 되어있는 경우 대부분 마스킹되어있다.
# 이름의 경우에는 마스킹되어있는 경우가 있고, 없는 경우가 있었다. ex) 유재* 유*현 유재현
# 링크의 경우 있는 경우도 있고, 없는 경우도 있다. 있는 경우엔 마스킹되지 않은 전체 http 주소

In [ ]:
# 데이터 받아오기

import pandas as pd
import re
from pykospacing import Spacing # 띄어쓰기 !pip install git+https://github.com/haven-jeon/PyKoSpacing.git
from hanspell import spell_checker

df = pd.read_csv("../smishing.csv")
spacing = Spacing()

# 맞춤법 교정 함수
def safe_spell_check(text):
    try:
        if isinstance(text, str) and 1 < len(text) <= 500:
            result = spell_checker.check(text)
            if hasattr(result, 'checked'):
                return result.checked
    except:
        pass
    return text

# 전처리 함수
def preprocess_text(text):
  
  ## 의미 없는 문자열 "ifg@" 제거
  text = text.replace("ifg@", "")

  ## url >> URL로 치환 | https나 http나 www로 시작하는 url 치환
  text = re.sub(r"(https?://|http?://|www\.)[^\s]+", "URL", text)

  ## 마스킹된 날짜를 DATE로 치환
  # yyyy년 MM월 dd일
  text = re.sub(r"\d{4}년\s?\d{1,2}월\s?\d{1,2}일", "DATE", text)

  # MM월 dd일
  text = re.sub(r"\d{1,2}월\s?\d{1,2}일", "DATE", text)

  # dd일만 나오는 경우 (예: "오늘 30일까지만 가능")
  text = re.sub(r"\d{1,2}일", "DATE", text)

  # yy년만 나오는 경우
  text = re.sub(r"\d{1,2}년", "DATE", text)

  # MM월만 나오는 경우
  text = re.sub(r"\d{1,2}월", "DATE", text)

  # yy년 MM월만 나오는 경우
  text = re.sub(r"\d{1,2}년\s?\d{1,2}월", "DATE", text)

  # MM/dd(일) 만 나오는 경우
  text = re.sub(r"\d{1,2}/\d{1,2}\(.\)", "DATE", text)

  # MM/dd(요일)
  text = re.sub(r"\d{1,2}/\d{1,2}\(\w+\)", "DATE", text)

  ## 마스킹된 시간을 TIME으로 치환
  # **:** 시:분 만 나오는 경우
  text = re.sub(r"\d{2}:\d{2}", "TIME", text)

  # *시 **시 *-*시간 *시간 *시~**시 **시-**시 **시 ~ **시 *시~**시 *시** *시-**시 **시~**시
  text = re.sub(r"\*{1,2}시", "TIME", text)
  text = re.sub(r"\*{1,2}시간", "TIME", text)
  text = re.sub(r"\*{1,2}시\s*[\-~]\s*\*{1,2}시", "TIME", text)

  # L*TT* >> LOTTO로 치환
  text = re.sub(r"L*TT*", "로또", text)

  # 이름 치환 ~회원님 ~님 ~ 회원님 ~ 님

 
  ## 전화번호 및 *로 마스킹된 숫자를 전부 NUM으로 치환
  # 연속된 *, 숫자, -, _, 공백, 쉼표가 포함된 마스킹 형태 전부를 하나의 'NUM'으로 통일
  text = re.sub(r"[\*\d,\-_\s]{6,}", "NUM", text)

  # 띄어쓰기 보정
  text = spacing(text)

  # 맞춤법 보정
  text = safe_spell_check(text)

  return text

# 배송은 포함하되, 국제 또는 국외가 없는 경우는 제거하기(유의미한 데이터가 아님)
condition = df["clean_df"].str.contains("배송") & ~df["clean_df"].str.contains("국제|국외")
df = df[~condition]
  
df["clean_df"] = df["CN"].apply(preprocess_text)

df